# Problema do Titanic do Kaggle
## ELT 574 - Aprendizado de Máquina
### Especialização em Inteligência Artificial e Computacional

#### Prof. Rodolpho Neves - NIAS/DEL/UFV

#### Problema:

O problema do desastre do **Titanic** é um clássico para quem está começando em **Machine Learning**. O objetivo é prever se um passageiro sobreviveu ou não ao naufrágio com base em algumas características. Dados baixados do Kaggle: [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic).

### Treinamento de um modelo de base (baseline model)

O treinamento do modelo de base seguirá as seguintes etapas:
1. Separação dos dados em treinamento e validação
2. Limpeza do banco de dados de treinamento e validação
3. Codificação das variáveis categóricas do problema
4. Ajuste de um modelo (RandomForest por simplicidade)
5. Comparação com os dados de validação 

Esperamos que este seja o primeiro modelo para que seja possível comparar estratégias de tratamento de dados. Para isso, vamos importar as bibliotecas a seguir.

#### 1. Separação dos dados em treinamento e validação

In [1211]:
# manipulacao de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# aprendizado de maquina e selecao de modelo
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [1212]:
dataset = pd.read_csv('kaggle-data/train.csv')
dataset.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [1213]:
colunas_principais = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = dataset.loc[:, colunas_principais]
y = dataset.loc[:, 'Survived']

X.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S


#### 2. Limpeza do banco de dados de treinamento e validação

In [1214]:
# separar o dataset em treino e validacao para ajustar o imputer
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train.shape, X_val.shape

((712, 7), (179, 7))

In [1215]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 834 to 522
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       578 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  711 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


Tratamento de variáveis categóricas

In [1216]:
# Imputacao de media e moda para valores faltantes em variáveis numéricas e categóricas
## instanciacao dos imputers
imputer_mean = SimpleImputer(strategy='mean')
imputer_mode = SimpleImputer(strategy='most_frequent', missing_values=np.nan)

## ajuste dos imputers
imputer_mean.fit(X_train[['Age']])
imputer_mode.fit(X_train[['Embarked']])

## transformacao dos dados
X_train[['Age']] = imputer_mean.transform(X_train[['Age']])
X_val[['Age']] = imputer_mean.transform(X_val[['Age']])

X_train[['Embarked']] = imputer_mode.transform(X_train[['Embarked']])
X_val[['Embarked']] = imputer_mode.transform(X_val[['Embarked']])

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 834 to 522
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  712 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


#### 3. Transformação das variáveis do problema

##### Variáveis numéricas

In [1217]:
# padronizacao das variáveis numéricas
variaveis_numericas = ['Age', 'SibSp', 'Parch', 'Fare']

# instanciar o scaler
scaler = StandardScaler()
scaler.fit(X_train.loc[:,variaveis_numericas])

# transformar os dados
X_train[variaveis_numericas] = scaler.transform(X_train.loc[:,variaveis_numericas])
X_val[variaveis_numericas] = scaler.transform(X_val.loc[:,variaveis_numericas])

##### Variáveis categóricas

In [1218]:
# ajustar o encoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = ['Pclass', 'Sex', 'Embarked']

encoder.fit(X_train.loc[:,encoded_features])
X_aux = pd.DataFrame(encoder.transform(X_train.loc[:,encoded_features]).toarray(), columns=encoder.get_feature_names_out(encoded_features))
X_train = pd.concat([X_train.drop(columns=encoded_features).reset_index(drop=True), X_aux], axis=1)
X_train.head(3)


X_aux = pd.DataFrame(encoder.transform(X_val.loc[:,encoded_features]).toarray(), columns=encoder.get_feature_names_out(encoded_features))
X_val = pd.concat([X_val.drop(columns=encoded_features).reset_index(drop=True), X_aux], axis=1)

X_val.head(3)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,-2.205638e-01,0.408726,-0.465857,-0.234782,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,3.079712e+00,-0.489462,-0.465857,0.024748,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,-2.664758e-16,-0.489462,-0.465857,-0.633936,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


#### 4. Ajuste de um modelo (RandomForest por simplicidade)

In [1219]:
# Treinando o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

#### 5. Comparação com os dados de validação 

In [1220]:
# Avaliação do modelo
y_pred = model.predict(X_val)
print(f'Acurácia: {accuracy_score(y_val, y_pred):.2f}')

Acurácia: 0.85


##### 6. Criar arquivo para submissão na competição

In [1221]:
# carregar os dados de teste
dataset_test = pd.read_csv('kaggle-data/test.csv')

# selecionar os dados de teste
colunas_principais = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_test = dataset_test.loc[:, colunas_principais]

# imputar os valores faltantes
X_test[['Age']] = imputer_mean.transform(X_test[['Age']])
X_test[['Embarked']] = imputer_mode.transform(X_test[['Embarked']])

# padronizar as variáveis numéricas
variaveis_numericas = ['Age', 'SibSp', 'Parch', 'Fare']
X_test[variaveis_numericas] = scaler.transform(X_test.loc[:,variaveis_numericas])

# aplicar o encoder
encoded_features = ['Pclass', 'Sex', 'Embarked']
X_aux = pd.DataFrame(encoder.transform(X_test.loc[:,encoded_features]).toarray(), columns=encoder.get_feature_names_out(encoded_features))
X_test = pd.concat([X_test.drop(columns=encoded_features).reset_index(drop=True), X_aux], axis=1)

# prever os resultados
y_pred = model.predict(X_test)

# salvar os resultados
dataset_test['Survived'] = y_pred
dataset_test[['PassengerId', 'Survived']].to_csv('kaggle-data/submission.csv', index=False)